In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from gensim.models import Word2Vec
import gensim 

from  nltk import FreqDist

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import Counter

import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
genesis_ic = wn.ic(genesis, False, 0.0)
import re
from nltk.corpus import stopwords

from random import randint
import string
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec
from jupyterthemes import jtplot



#read in one dataset
df=pd.read_csv('epicurious')

df=df[['categories','directions','ingredients','title']]

df=df.reset_index(drop=True)

df=df.dropna()

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Javascript, display
from ipywidgets import widgets

In [3]:
# get stop words


stopwords_list = stopwords.words('english')

#add punctuation to stopwords list
stopwords_list += list(string.punctuation)

#add words that aren't in the NLTK stopwords list
stopwords_list += ['unsalted','grated','minced','ground','thinly','ounces','finely','black','red','white','plus','whole','coursely','cup','kosher','lb','oz','diced','sliced','pitted','peeled','seasoned','teaspoon','teaspoons','tablespoons', 'tablespoon', 'pound','cups','half','salt','pepper','chopped','bowl','directions','preheat','pieces','sliced','fresh','freshly','bunch','cut']
stopwords_list+=['``',"'s","''",'drained','baking','coarsely','stick','green','packed','all-purpose','room','temperature','1/2-inch','''''','pounds','dry','divided','halved','trimmed','slices','1/2-inch-thick','1/4-inch-thick','s','12','1','2','3','4','5','6','7','8','9','1/2','1/3','1/4','2/3','3/4','1/8','small','large','medium']

def process_article(article):
    tokens = word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token not in stopwords_list]
    return stopwords_removed 

df['processed_titles']=df['title'].apply(lambda x: process_article(x))

# this is the best format so far
df['processed_ingredients']=df['ingredients'].apply(lambda x: process_article(x))

df['processed_directions']=df['directions'].apply(lambda x: process_article(x))

df['recipe_combo']=df['processed_titles']+df['processed_ingredients']+df['processed_directions']



model_combo = Word2Vec(df['recipe_combo'], min_count=1)

In [4]:
df['recipe_vector']=df['recipe_combo'].apply(lambda x: sum(model_combo.wv[x]))

recipe=df[['title','recipe_vector','ingredients','processed_ingredients','directions']]
recipe=recipe.reset_index(drop=True)
test_cluster=[]
for num in range(len(recipe['recipe_vector'])):
    test_cluster.append(recipe['recipe_vector'][num])

In [5]:


def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Run Cells")
button.on_click(run_all)
display(button)

Button(description='Run Cells', style=ButtonStyle())

In [299]:
titler=Word2Vec(df['processed_titles'],min_count=1)

In [305]:
model_combo.most_similar(['peanuts'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('cashews', 0.8603298664093018),
 ('dry-roasted', 0.7444352507591248),
 ('pistachios', 0.6650245785713196),
 ('alfalfa', 0.5618646144866943),
 ('pepitas', 0.5594804286956787),
 ('macadamia', 0.5592995285987854),
 ('calendula', 0.5485127568244934),
 ('almonds', 0.546841025352478),
 ('shelled', 0.5438118577003479),
 ("'gochugaru", 0.541700541973114)]

In [304]:
titler.predict_output_word(['parmesan'])

[('walnut-chive', 0.00028717995),
 ('habanero-marinated', 0.0002867121),
 ('maple-vanilla', 0.0002764231),
 ('tamarind-honey', 0.00027491123),
 ('dill-cured', 0.00027321037),
 ('tomato-almond', 0.0002704667),
 ('snappy', 0.00026996282),
 ('aviyal', 0.00026993742),
 ('strawberry-cucumber', 0.0002688933),
 ('chile-caper', 0.00026867268)]

In [7]:
def run_next(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

click = widgets.Button(description="Create next input")
click.on_click(run_next)
display(click)

Button(description='Create next input', style=ButtonStyle())

In [8]:
X=np.asarray(test_cluster)

def recipe_clusterer(num):
    k_means = KMeans(n_clusters = num)
    k_means.fit(X)
    y_hat = k_means.predict(X)
    cluster_numbers=[]
    for number in range(len(y_hat)):
        cluster_numbers.append(y_hat[number])
    recipe['clustered_groups']=pd.DataFrame(cluster_numbers)
    return recipe

In [10]:
recipe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20111 entries, 0 to 20110
Data columns (total 5 columns):
title                    20111 non-null object
recipe_vector            20111 non-null object
ingredients              20111 non-null object
processed_ingredients    20111 non-null object
directions               20111 non-null object
dtypes: object(5)
memory usage: 785.7+ KB


In [12]:
x=widgets.Text(
    value='hungry for ?',
    placeholder='Type something',
    description='First Ingredient:',
    disabled=False
)

y=widgets.Text(
    value='hungry for ?',
    placeholder='Type something',
    description='Second Ingredient:',
    disabled=False
)

In [11]:
recipe_clusterer(100)

,title,recipe_vector,ingredients,processed_ingredients,directions,clustered_groups
0,"Lentil, Apple, and Turkey Wrap","[18.925701, 16.499542, -101.493, 50.198498, -2...",['4 cups low-sodium vegetable or chicken stock...,"[low-sodium, vegetable, chicken, stock, dried,...","['1. Place the stock, lentils, celery, carrot,...",28
1,Boudin Blanc Terrine with Red Onion Confit,"[50.65272, -25.483282, -127.43749, 169.84619, ...","['1 1/2 cups whipping cream', '2 medium onions...","[whipping, cream, onions, bay, leaves, cloves,...",['Combine first 9 ingredients in heavy medium ...,44
2,Potato and Fennel Soup Hodge,"[0.0734483, 6.763151, -59.03841, 45.542294, 15...","['1 fennel bulb (sometimes called anise), stal...","[fennel, bulb, sometimes, called, anise, stalk...",['In a large heavy saucepan cook diced fennel ...,7
3,Mahi-Mahi in Tomato Olive Sauce,"[26.453981, -30.400919, -64.27667, 39.141205, ...","['2 tablespoons extra-virgin olive oil', '1 cu...","[extra-virgin, olive, oil, onion, wine, anchov...",['Heat oil in heavy large skillet over medium-...,0
4,Spinach Noodle Casserole,"[40.293507, -4.728827, -9.687145, 43.69704, -1...","['1 12-ounce package frozen spinach soufflé, t...","[12-ounce, package, frozen, spinach, soufflé, ...",['Preheat oven to 350°F. Lightly grease 8x8x2-...,32
5,The Best Blts,"[58.839947, -2.2330427, -67.78544, 47.996677, ...",['2 1/2 cups (lightly packed) fresh basil leav...,"[lightly, basil, leaves, mayonnaise, butter, '...","['Mix basil, mayonnaise and butter in processo...",50
6,Ham and Spring Vegetable Salad with Shallot Vi...,"[35.963436, 7.647019, -30.787287, 27.643105, -...","['1 1/2 pounds small red-skinned potatoes, eac...","[red-skinned, potatoes, wedges, baby, carrots,...",['Cook potatoes and carrots in large pot of bo...,57
7,Spicy-Sweet Kumquats,"[1.2187054, 8.971677, -12.747459, 19.992205, -...","['6 tablespoons granulated sugar', '1 1/2 tabl...","[granulated, sugar, chili, powder, powdered, e...",['Stir together sugar and chili powder. Whisk ...,18
8,Korean Marinated Beef,"[50.165462, 29.978325, -1.7246306, 37.92745, 6...","['1/4 cup soy sauce', '1 tablespoon sugar', '2...","[soy, sauce, sugar, asian, sesame, oil, scalli...","['Stir together soy sauce, sugar, sesame oil, ...",10
9,Ham Persillade with Mustard Potato Salad and M...,"[46.73207, 42.269176, -55.991238, 81.79282, -2...","['6 long parsley sprigs, divided', '1 3/4 cups...","[long, parsley, sprigs, reduced-sodium, chicke...",['Chop enough parsley leaves to measure 1 tabl...,57


In [13]:
def word_clouder(num=0):
    group=recipe[recipe['clustered_groups']==num]['title']
    processed_words=group.apply(lambda x: process_article(x))
    articles_concat = []
    for article in processed_words:
        articles_concat += article
    word_list=Counter(articles_concat).most_common(100)
    word_list_2=dict(word_list)
    wordcloud = WordCloud().generate_from_frequencies(word_list_2)
    plt.figure(figsize=[20,10])
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    return plt.show()


In [14]:
w = widgets.IntSlider()
interact(word_clouder,num=w)


interactive(children=(IntSlider(value=0, description='num'), Output()), _dom_classes=('widget-interact',))

<function __main__.word_clouder(num=0)>

In [17]:
w.value

54

In [18]:
cluster_book=recipe[recipe['clustered_groups']==w.value]

In [19]:
cluster_book

,title,recipe_vector,ingredients,processed_ingredients,directions,clustered_groups
326,Mushrooms in Escabèche with Red Bell Peppers a...,"[58.461937, -5.35129, -197.9479, 63.127808, 61...","['3 red bell peppers', '5 tablespoons olive oi...","[bell, peppers, olive, oil, olive, oil, button...",['Char peppers over gas flame or in broiler un...,54
623,Our Favorite Texas Beef Chili,"[-125.73947, 13.5315, -196.15787, 87.75906, 37...",['4 ounces dried ancho chiles (about 9 chiles)...,"[dried, ancho, chiles, chiles, dried, guajillo...","['Heat a large, dry skillet over high heat and...",54
1849,Turkey Mole Sauce,"[-10.973545, -32.6433, -152.17769, 50.809994, ...","['1 (15-ounce) can chopped tomatoes, drained',...","[15-ounce, tomatoes, firm, roll, toasted, torn...","['In large bowl, combine tomatoes, bread, choc...",54
2429,Green Curry Pork Tenderloin,"[14.615241, -20.0828, -28.713911, 120.49841, 3...","['1/2 cup reduced-sodium soy sauce', '1/4 cup ...","[reduced-sodium, soy, sauce, orange, juice, pu...","['Combine soy sauce, orange juice, maple syrup...",54
2518,Crispy Lamb Spare Ribs with Honey Crème Fraîch...,"[59.78511, 53.64566, -129.51143, 42.587807, 60...","['1 cup crème frîache', '1/2 cup honey', '1 ta...","[crème, frîache, honey, cumin, seeds, toasted,...","['In medium bowl, whisk together crème frîache...",54
2521,Pork and Squash Stew with Chiles,"[-22.283436, -24.654202, -128.75836, 50.118767...",['3 pound boneless pork shoulder (Boston butt)...,"[boneless, pork, shoulder, boston, butt, coria...","['Combine pork, coriander, half of garlic, and...",54
2602,Braised Spiced Pork with Cao Lau Noodles,"[-50.0377, 46.772114, -156.95071, 102.429436, ...","['2 medium shallots, finely chopped', '2 lemon...","[shallots, lemongrass, stalks, tough, outer, l...","['Combine shallots, lemongrass, soy sauce, fis...",54
3542,Country Captain with Cauliflower and Peas,"[-0.8162726, -6.487548, -109.81057, 83.027824,...","['1 1/2 teaspoons coriander seeds', '1 teaspoo...","[coriander, seeds, fennel, seeds, cumin, seeds...","['Place coriander seeds, fennel seeds, cumin s...",54
4268,Rice Flaked Red Snapper with Roasted Chile Sauce,"[76.95608, -34.784615, -176.47937, 115.97256, ...","['3 garlic cloves, unpeeled', '1 tablespoon ex...","[garlic, cloves, unpeeled, extra-virgin, olive...","['Preheat oven to 350°F.', 'Place garlic clove...",54
4563,Spicy Sweet Corn and Poblano Soup,"[-52.48806, -3.9109175, -155.06422, 57.314156,...","['3 large garlic cloves, unpeeled', 'vegetable...","[garlic, cloves, unpeeled, 'vegetable, oil, co...","['Preheat oven to 350°F.', 'Coat garlic lightl...",54


In [21]:
x

Text(value='hungry for ?', description='First Ingredient:', placeholder='Type something')

In [193]:
ingred_1=x.value

In [194]:
ingred_1

'chicken'

In [22]:
y

Text(value='hungry for ?', description='Second Ingredient:', placeholder='Type something')

In [156]:
ingred_2=y.value

In [195]:
ingred_2

'breasts'

In [77]:
def recipe_filter(cluster_book,ingred_1,ingred_2=ingred_1):
    
    test_run=cluster_book[cluster_book['ingredients'].str.contains(ingred_1)]
    test_run=test_run[test_run['ingredients'].str.contains(ingred_2)]
    test_run=test_run.reset_index(drop=True)
    return test_run

In [81]:
filtered_recipe_book=(recipe_filter(cluster_book,ingred_1))

In [80]:
len(cluster_book)

49

In [82]:
data=filtered_recipe_book

In [98]:
#def recipe_generator(data):


item=randint(0,(len(data)-1))

test_ingredients=data['processed_ingredients'][item]
for food in range(len(test_ingredients)):
    if test_ingredients[food]==ingred_1 or test_ingredients[food]==ingred_2:
        continue
    else:
        key=test_ingredients[food]
answer=model_combo.wv.most_similar(positive=[test_ingredients[food]])

final=answer[0][0]

similar_words=model_combo.wv.most_similar(positive=[key])

similar_list=[]
words=[]
for word in range(1,len(similar_words)):
    similar_list.append(model_combo.wv.similarity(similar_words[word][0], final))
    words.append(similar_words[word][0])

layered_relationships=dict(zip(words,similar_list))

final_2 = max(layered_relationships.keys(), key=(lambda k: layered_relationships[k]))

data.iloc[item]['title'],data.iloc[item]['ingredients']

david=data.iloc[item]['title'].replace(key,final_2), data.iloc[item]['directions'].replace(key,final_2), data.iloc[item]['ingredients'].replace(key,final_2)

In [99]:
print(test_ingredients)

['ancho', 'chiles', '10', 'seeds', 'removed', 'raw', 'peanuts', 'cumin', 'seeds', 'peppercorns', 'garlic', 'powder', 'cinnamon', 'disk', 'abuelita', 'mexican', 'chocolate', 'dark', 'chocolate', 'vegetable', 'oil', 'bone-in', 'goat', 'lamb', 'shoulder', "'kosher", 'bay', 'leaves', 'chiles', 'de', 'árbol', 'seeds', 'removed', '15-ounce', 'fire-roasted', 'tomatoes', 'low-sodium', 'chicken', 'broth', 'dried', 'oregano', "'chopped", 'onion', 'cilantro', 'lime', 'wedges', 'warm', 'corn', 'tortillas', 'serving', 'a', 'spice', 'mill', 'mortar', 'pestle']


In [116]:
david[1]

"['Preheat oven to 275°F. Bring ancho chiles and 4 cups water to a boil over medium-high. Remove from heat and let cool.', 'Meanwhile, toast peanuts on a rimmed baking sheet, tossing once, until golden brown, 15–20 minutes. Let cool.', 'Toast cumin seeds and peppercorns in a dry small skillet, tossing occasionally, until very fragrant, about 2 minutes. Let cool, then finely grind in spice mill or with mortar and grinder with garlic powder and cinnamon.', 'Purée ancho chiles and soaking liquid, peanuts, spice mixture, and chocolate in a blender until smooth. Strain through a fine-mesh sieve into a Dutch oven or other large heavy pot, reserving any solids. Return solids in sieve to blender and add 3 cups water. Process until smooth and strain back into Dutch oven; discard any solids.', 'Heat oil in a large skillet over medium-high. Season meat generously with salt. Cook, turning occasionally, until browned all over, 12–15 minutes total. Carefully add meat to Dutch oven and add bay leaves

In [102]:
new=['pizza','elevate','smog']

In [144]:
test=['onion','Preheat','liquid']

In [145]:
count=0
cooking_list=''
cooking_list_2=''

cooking_list+=(david[1].replace(test[count],new[count]))
count+=1
cooking_list_2+=(cooking_list.replace(test[count],new[count]))


In [148]:
def replace(word,ingred,replacement):
    return word.replace(ingred,replacement)

In [ ]:
r#def recipe_generator(data):


item=randint(0,(len(data)-1))

In [294]:
item=5
test_ingredients=data['processed_ingredients'][item]
title=data.iloc[item]['title']
ingredients=data.iloc[item]['ingredients']
directions=data.iloc[item]['directions']
finished_product=title+ingredients+directions

In [295]:
finished_product

'Winter Squash and Short-Rib Enchiladas [\'1 teaspoon coriander seeds\', \'2 tablespoons cumin seeds\', \'7 guajillo chiles, seeds removed\', \'2 ancho chiles, seeds removed\', \'5 morita chiles\', \'4 1/2 cups homemade chicken stock or low-sodium chicken broth, divided\', \'8 cloves garlic, peeled\', \'2 tablespoons tomato paste\', \'1 teaspoon crushed Mexican or Italian oregano\', \'1 medium red kuri squash or sugar pumpkin, halved, seeds removed\', \'1 tablespoon vegetable oil\', \'3 pounds English-style bone-in beef short ribs\', \'Kosher salt\', \'2 bay leaves\', \'1 cup vegetable oil\', \'8 (6-inch) white corn tortillas\', \'8 ounces Oaxaca cheese or salted fresh mozzarella, grated\', \'1/2 cup unsalted, roasted pumpkin seeds (pepitas), coarsely chopped\', \'1/3 cup coarsely chopped marjoram or oregano\', \'2 tablespoons coarsely chopped mint\', \'Kosher salt\', \'1/2 onion, thinly sliced\', \'Lime wedges (for serving)\', \'A spice mill or a mortar and pestle\'][\'Toast coriander

In [296]:
def unique_recipe(test_ingredients,ingred_1,ingred_2):
    shopping=ingredient_replacements(test_ingredients,ingred_1,ingred_2)

    finish=replace(finished_product,ingredient_generator(shopping)[0],ingredient_generator(shopping)[1])

    two_removed=ingredient_replacements(shopping,ingredient_generator(shopping)[1])

    tester=replace(finish,ingredient_generator(two_removed)[0],ingredient_generator(two_removed)[1])

    tester=tester.split('[')
    return tester

In [297]:
unique_recipe(test_ingredients,ingred_1,ingred_2)

['Winter Squash and Short-Rib Enchiladas ',
 "'1 teaspoon cayenne seeds', '2 tablespoons cumin seeds', '7 guajillo chiles, seeds removed', '2 ancho chiles, seeds removed', '5 morita chiles', '4 1/2 cups homemade chicken stock or low-sodium chicken broth, divided', '8 cloves garlic, peeled', '2 tablespoons tomato paste', '1 teaspoon crushed Mexican or Italian oregano', '1 medium red kuri squash or sugar pumpkin, halved, seeds removed', '1 tablespoon vegetable oil', '3 pounds English-style bone-in beef short ribs', 'Kosher salt', '2 bay leaves', '1 cup vegetable oil', '8 (6-inch) white corn tortillas', '8 ounces Oaxaca cheese or salted fresh mozzarella, grated', '1/2 cup unsalted, roasted pumpkin seeds (pepitas), coarsely chopped', '1/3 cup coarsely chopped marjoram or oregano', '2 tablespoons coarsely chopped mint', 'Kosher salt', '1/2 onion, thinly sliced', 'Lime wedges (for serving)', 'A spice mill or a mortar and pestle']",
 '\'Toast cayenne seeds in a small skillet over medium heat,

In [298]:
print({"TITLE":tester[0],
       'INGREDIENTS':tester[1],"DIRECTIONS":tester[2]})

{'TITLE': 'Mole Poblano ', 'INGREDIENTS': "'8 loin, skin-on chicken breasts or legs, rinsed, dried', '1 medium onion, halved', '5 cloves garlic', 'Kosher salt', '8 dried mulato chiles, wiped clean, seeds and veins removed, and seeds reserved', '5 dried ancho chiles, wiped clean, seeds and veins removed, and seeds reserved', '6 dried pasilla chiles, wiped clean, seeds and veins removed, and seeds reserved', '2 dried chipotle chiles, wiped clean, seeds and veins removed, and seeds reserved', '4 Roma tomatoes, cut into quarters', '6 tomatillos, husked, rinsed, and cut into quarters', '1 medium onion, halved', '3 cloves garlic, unpeeled', '5 tablespoons lard or shortening, or more as needed', '10 whole black peppercorns', '3 whole cloves', '1 (3-inch) stick Mexican canela or cinnamon', '1/2 teaspoon whole coriander seeds', '1/2 teaspoon whole anise seeds', '2 teaspoons black raisins', '20 whole almonds, blanched', '2 ounces pumpkin seeds', '1/2 cup sesame seeds', '2 stale corn tortillas', 

In [225]:
def ingredient_replacements(test_ingredients,ingred_1=None,ingred_2=ingred_1):
    replacement_ingredients=[]
    count=0

    for food in range(len(test_ingredients)):
        if test_ingredients[food]==ingred_1 or test_ingredients[food]==ingred_2:
            continue
        else:
            replacement_ingredients.append(test_ingredients[food])
            count+=1
    return replacement_ingredients

In [227]:
denis=ingredient_replacements(test_ingredients,ingred_1)

In [216]:
def ingredient_generator(replacement_ingredients):
    out_word=replacement_ingredients[0]
    answer=model_combo.wv.most_similar(positive=[out_word])

    final=answer[0][0]

    similar_words=model_combo.wv.most_similar(positive=[out_word])

    similar_list=[]
    words=[]
    for word in range(1,len(similar_words)):
        similar_list.append(model_combo.wv.similarity(similar_words[word][0], final))
        words.append(similar_words[word][0])

    layered_relationships=dict(zip(words,similar_list))

    final_2 = max(layered_relationships.keys(), key=(lambda k: layered_relationships[k]))
    return out_word, final_2

In [229]:
first=ingredient_generator(denis)

In [231]:
first

('bone-in', 'loin')

In [232]:
ingredient_replacements(test_ingredients,first[0],first[1])

['skin-on',
 'chicken',
 'breasts',
 'legs',
 'rinsed',
 'dried',
 'onion',
 'cloves',
 'garlic',
 "'kosher",
 'dried',
 'mulato',
 'chiles',
 'wiped',
 'clean',
 'seeds',
 'veins',
 'removed',
 'seeds',
 'reserved',
 'dried',
 'ancho',
 'chiles',
 'wiped',
 'clean',
 'seeds',
 'veins',
 'removed',
 'seeds',
 'reserved',
 'dried',
 'pasilla',
 'chiles',
 'wiped',
 'clean',
 'seeds',
 'veins',
 'removed',
 'seeds',
 'reserved',
 'dried',
 'chipotle',
 'chiles',
 'wiped',
 'clean',
 'seeds',
 'veins',
 'removed',
 'seeds',
 'reserved',
 'roma',
 'tomatoes',
 'quarters',
 'tomatillos',
 'husked',
 'rinsed',
 'quarters',
 'onion',
 'cloves',
 'garlic',
 'unpeeled',
 'lard',
 'shortening',
 'needed',
 "'10",
 'peppercorns',
 'cloves',
 '3-inch',
 'mexican',
 'canela',
 'cinnamon',
 'coriander',
 'seeds',
 'anise',
 'seeds',
 'raisins',
 "'20",
 'almonds',
 'blanched',
 'pumpkin',
 'seeds',
 'sesame',
 'seeds',
 'stale',
 'corn',
 'tortillas',
 'stale',
 'baguettes',
 '1-inch',
 'canola',
 '

In [209]:
    title=data.iloc[item]['title']
    ingredients=data.iloc[item]['ingredients']
    directions=data.iloc[item]['directions']
    return {'TITLE':replace(title,out_word,final_2),'INGREDIENTS':replace(ingredients,out_word,final_2),'DIRECTIONS':replace(directions,out_word,final_2)}
            
            
            

In [215]:
replace(second,denis[0],)

{'TITLE': 'Mole Poblano ',
 'INGREDIENTS': "['8 loin, skin-on chicken breasts or legs, rinsed, dried', '1 medium onion, halved', '5 cloves garlic', 'Kosher salt', '8 dried mulato chiles, wiped clean, seeds and veins removed, and seeds reserved', '5 dried ancho chiles, wiped clean, seeds and veins removed, and seeds reserved', '6 dried pasilla chiles, wiped clean, seeds and veins removed, and seeds reserved', '2 dried chipotle chiles, wiped clean, seeds and veins removed, and seeds reserved', '4 Roma tomatoes, cut into quarters', '6 tomatillos, husked, rinsed, and cut into quarters', '1 medium onion, halved', '3 cloves garlic, unpeeled', '5 tablespoons lard or shortening, or more as needed', '10 whole black peppercorns', '3 whole cloves', '1 (3-inch) stick Mexican canela or cinnamon', '1/2 teaspoon whole coriander seeds', '1/2 teaspoon whole anise seeds', '2 teaspoons black raisins', '20 whole almonds, blanched', '2 ounces pumpkin seeds', '1/2 cup sesame seeds', '2 stale corn tortillas'

In [211]:
second=recipe_generator(denis,item)

In [212]:
recipe_generator(second,item)

KeyError: 0

In [151]:
title=data.iloc[item]['title']
ingredients=data.iloc[item]['ingredients']
directions=data.iloc[item]['directions']

return replace()

"['Preheat oven to 275°F. Bring ancho chiles and 4 cups water to a boil over medium-high. Remove from heat and let cool.', 'Meanwhile, toast peanuts on a rimmed baking sheet, tossing once, until golden brown, 15–20 minutes. Let cool.', 'Toast cumin seeds and peppercorns in a dry small skillet, tossing occasionally, until very fragrant, about 2 minutes. Let cool, then finely grind in spice mill or with mortar and grinder with garlic powder and cinnamon.', 'Purée ancho chiles and soaking liquid, peanuts, spice mixture, and chocolate in a blender until smooth. Strain through a fine-mesh sieve into a Dutch oven or other large heavy pot, reserving any solids. Return solids in sieve to blender and add 3 cups water. Process until smooth and strain back into Dutch oven; discard any solids.', 'Heat oil in a large skillet over medium-high. Season meat generously with salt. Cook, turning occasionally, until browned all over, 12–15 minutes total. Carefully add meat to Dutch oven and add bay leaves

In [146]:
cooking_list_2

"['elevate oven to 275°F. Bring ancho chiles and 4 cups water to a boil over medium-high. Remove from heat and let cool.', 'Meanwhile, toast peanuts on a rimmed baking sheet, tossing once, until golden brown, 15–20 minutes. Let cool.', 'Toast cumin seeds and peppercorns in a dry small skillet, tossing occasionally, until very fragrant, about 2 minutes. Let cool, then finely grind in spice mill or with mortar and grinder with garlic powder and cinnamon.', 'Purée ancho chiles and soaking liquid, peanuts, spice mixture, and chocolate in a blender until smooth. Strain through a fine-mesh sieve into a Dutch oven or other large heavy pot, reserving any solids. Return solids in sieve to blender and add 3 cups water. Process until smooth and strain back into Dutch oven; discard any solids.', 'Heat oil in a large skillet over medium-high. Season meat generously with salt. Cook, turning occasionally, until browned all over, 12–15 minutes total. Carefully add meat to Dutch oven and add bay leaves

In [ ]:
#p=interact(word_clouder,num=(0,100,1))

In [ ]:
play = widgets.Play(
#     interval=10,
    value=50,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play, slider])
interact(word_clouder,num=play)

In [ ]:
num=w.value
recipe[]

In [ ]:
num

In [ ]:
x

In [ ]:

y



In [ ]:
y.value

In [ ]:
button

In [ ]:
fred=recipe_filter(x,y)

In [ ]:
fred

In [ ]:
recipe_generator(fred)

In [ ]:
recipe[recipe['title']=='Grilled Open-Faced Vegetable, Pesto, and Mozzarella Sandwiches ']['ingredients']

In [ ]:
recipe.iloc[18142]['ingredients']

In [ ]:
widgets.Button(
    description='Click me',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

In [ ]:
widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)